###############################################################################
# SRSWTIZeroShot
###############################################################################

In [2]:
# ---------------------------  Imports -------------------------------
from srswti_axis import SRSWTIZeroShot
from utils import call_groq_llm, SAMPLE_DOCS
import random


/home/zhreyas/miniconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-10 14:13:32.655324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741596212.674809  835955 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741596212.680222  835955 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 14:13:32.701035: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-c

# --------------------------- Basic  Demos -------------------------

In [3]:
# Initialize the zero-shot classifier
zero_clf = SRSWTIZeroShot() 

INFO:SRSWTI-ZeroShot:Initializing SRSWTI Zero-Shot Classifier on cuda with batch size None


Initializing Zero-Shot Classifier on cuda with batch size None

Device set to use cuda


In [4]:
# 1) Basic single-label classification
text_1 = "The new AI model is highly efficient in diagnosing lung cancer."
labels_1 = ["medical", "finance", "technology"]
res_1 = zero_clf.classify_text(text_1, labels_1, multi_label=False)
print("== ZeroShot Example 1: Single-label ==")
print(res_1, "\n")


INFO:SRSWTI-ZeroShot:Classification completed in 0.3586 seconds


== ZeroShot Example 1: Single-label ==
{'technology': 0.7873777151107788, 'medical': 0.2109856754541397, 'finance': 0.001636624918319285} 



In [5]:
# 2) Multi-label classification with domain-specific categories
text_2 = "We used blockchain to secure medical records and reduce cost inefficiency."
labels_2 = ["healthcare", "blockchain", "cost_reduction", "product_review"]
res_2 = zero_clf.classify_text(text_2, labels_2, multi_label=True)
print("== ZeroShot Example 2: Multi-label ==")
print(res_2, "\n")


INFO:SRSWTI-ZeroShot:Classification completed in 0.0789 seconds


== ZeroShot Example 2: Multi-label ==
{'blockchain': 0.9970793724060059, 'cost_reduction': 0.9904685616493225, 'healthcare': 0.9878616333007812, 'product_review': 0.019475707784295082} 



In [6]:
# 3) Zero-Shot in a RAG-like scenario
retrieved_doc_snippet = "Ethereum smart contracts improved supply chain operations for hospitals."
labels_3 = ["medical", "blockchain", "supply_chain", "education"]
res_3 = zero_clf.classify_text(retrieved_doc_snippet, labels_3, multi_label=True)
print("== ZeroShot Example 3 (RAG context) ==")
print(res_3, "\n")

INFO:SRSWTI-ZeroShot:Classification completed in 0.0760 seconds


== ZeroShot Example 3 (RAG context) ==
{'medical': 0.9972496628761292, 'supply_chain': 0.8747292757034302, 'blockchain': 0.7565616965293884, 'education': 4.034030280308798e-05} 



# ------------------- LLM + RAG Demos ----------------------

In [7]:
def rag_demo_a(user_query, docs=SAMPLE_DOCS):
    """
    1) We pick a doc (e.g. random) from docs.
    2) Zero-shot classify it.
    3) Feed classification + doc snippet to Groq LLM for explanation.
    """
    if not docs:
        print("[RAG Demo A] No docs loaded.")
        return

    chosen_doc = random.choice(docs)
    print(f"[RAG Demo A] Chosen doc for '{user_query}':", chosen_doc)

    labels_demo = ["neural networks", "classical ml", "blockchain", "quantum"]
    classification = zero_clf.classify_text(chosen_doc, labels_demo, multi_label=True)

    prompt = (
        f"User asked: '{user_query}'\n\n"
        f"Relevant doc snippet:\n{chosen_doc}\n\n"
        f"Classification: {classification}\n"
        "Provide a short explanation."
    )
    llm_response = call_groq_llm(prompt)
    print(llm_response, "\n")

def rag_demo_b(user_query, docs=SAMPLE_DOCS):
    """
    1) Classify each doc with zero-shot.
    2) Filter docs that have label 'blockchain' > 0.5
    3) Summarize or respond with Groq LLM.
    """
    if not docs:
        print("[RAG Demo B] No docs loaded.")
        return

    matching_docs = []
    for d in docs:
        labels_demo = ["blockchain", "medical", "ai", "quantum"]
        class_res = zero_clf.classify_text(d, labels_demo, multi_label=True)
        if class_res.get("blockchain", 0) > 0.5:
            matching_docs.append(d)

    combined_info = "\n".join(matching_docs) if matching_docs else "No blockchain doc found."
    prompt = (
        f"User asked: '{user_query}'\n\n"
        f"Found docs labeled 'blockchain' above 0.5:\n{combined_info}\n"
        "Explain how this is relevant to the query."
    )
    llm_response = call_groq_llm(prompt)
    print("[RAG Demo B] Matching docs:", matching_docs)
    print(llm_response, "\n")




In [8]:
rag_demo_a("Any references to AI solutions?", SAMPLE_DOCS)

[RAG Demo A] Chosen doc for 'Any references to AI solutions?': Quantum computing represents a revolutionary approach to computation that harnesses the principles of quantum mechanics. Unlike classical computers that use bits (0s and 1s), quantum computers utilize quantum bits or qubits that can exist in multiple states simultaneously through superposition. This fundamental difference enables quantum computers to perform certain calculations exponentially faster than their classical counterparts, particularly in areas like factoring large numbers, searching unsorted databases, and simulating quantum systems.

The potential impact of quantum computing extends across numerous fields, including cryptography, drug discovery, materials science, and artificial intelligence. In cryptography specifically, algorithms like Shor's could theoretically break widely-used encryption schemes that rely on the difficulty of factoring large prime numbers. This has spurred the development of post-quantum c

INFO:SRSWTI-ZeroShot:Classification completed in 0.2228 seconds


[Groq LLM] There is a reference to artificial intelligence (AI) in the document, but it is brief. The text mentions that quantum computing has the potential to impact numerous fields, including artificial intelligence. However, it does not provide any further details on how quantum computing relates to AI or what specific AI solutions are being developed. The classification results also show a relatively low score for 'classical ml' (0.0478) and 'neural networks' (0.2297), suggesting that AI is not a primary focus of the document. 



In [9]:
rag_demo_b("Any references to blockchain?", SAMPLE_DOCS)


INFO:SRSWTI-ZeroShot:Classification completed in 0.1802 seconds
INFO:SRSWTI-ZeroShot:Classification completed in 0.1769 seconds
INFO:SRSWTI-ZeroShot:Classification completed in 0.2158 seconds


[RAG Demo B] Matching docs: ['## Generative AI and Blockchain Integration\n\n### 1. NFT Creation and Verification\n- Generative AI creates unique digital assets that can be minted as NFTs\n- Smart contracts can verify the authenticity and provenance of AI-generated art\n\n### 2. Decentralized AI Training\n- Blockchain-based platforms enable distributed training of AI models\n- Token incentives for contributing computing resources or training data\n\n### 3. On-chain AI Models\n- AI models deployed directly on blockchain networks\n- Enables transparent, verifiable AI inference with immutable records\n\n### 4. Governance and Data Marketplaces\n- DAOs (Decentralized Autonomous Organizations) for governing AI systems\n- Decentralized data marketplaces for training AI with fair compensation to data owners\n\n### 5. Challenges and Considerations\n- Computational limitations of current blockchain infrastructure\n- Privacy concerns when combining transparent ledgers with AI capability\n- Regula